In [1]:
from openai import OpenAI
import numpy as np

In [11]:
client = OpenAI(api_key="")

In [4]:
with open('nba.txt', 'r') as f:
    text = f.read()
    print(text)

About the NBA
The National Basketball Association, or NBA, is a professional basketball league comprised of 30 teams across North America featuring the best basketball players in the world.

FREQUENTLY ASKED QUESTIONS

How many teams are there in the NBA?

The NBA is a collection of 30 teams, broken down by two conferences with three divisions apiece. The Western Conference plays host to the Northwest, Pacific and Southwest Divisions, featuring the following 15 teams:

Northwest: Denver Nuggets, Minnesota Timberwolves, Oklahoma City Thunder, Portland Trail Blazers, Utah Jazz
Pacific: Golden State Warriors, L.A. Clippers, Los Angeles Lakers, Phoenix Suns, Sacramento Kings
Southwest: Dallas Mavericks, Houston Rockets, Memphis Grizzlies, New Orleans Pelicans, San Antonio Spurs
The Eastern Conference’s 15 squads also break up across three divisions: the Atlantic, Southeast and Central:

Atlantic: Boston Celtics, Brooklyn Nets, New York Knicks, Philadelphia 76ers, Toronto Raptors
Central: C

In [5]:
CHUNNK_SIZE = 100
chunks = []
current_place = 0
while current_place < len(text):
    current_chunk = text[current_place : current_place + CHUNNK_SIZE]
    chunks.append(current_chunk)
    current_place += CHUNNK_SIZE
print(chunks)

['About the NBA\nThe National Basketball Association, or NBA, is a professional basketball league compr', 'ised of 30 teams across North America featuring the best basketball players in the world.\n\nFREQUENTL', 'Y ASKED QUESTIONS\n\nHow many teams are there in the NBA?\n\nThe NBA is a collection of 30 teams, broken', ' down by two conferences with three divisions apiece. The Western Conference plays host to the North', 'west, Pacific and Southwest Divisions, featuring the following 15 teams:\n\nNorthwest: Denver Nuggets,', ' Minnesota Timberwolves, Oklahoma City Thunder, Portland Trail Blazers, Utah Jazz\nPacific: Golden St', 'ate Warriors, L.A. Clippers, Los Angeles Lakers, Phoenix Suns, Sacramento Kings\nSouthwest: Dallas Ma', 'vericks, Houston Rockets, Memphis Grizzlies, New Orleans Pelicans, San Antonio Spurs\nThe Eastern Con', 'ference’s 15 squads also break up across three divisions: the Atlantic, Southeast and Central:\n\nAtla', 'ntic: Boston Celtics, Brooklyn Nets, New York Kn

In [12]:
embedded_chunks = [] #-> [ ('About the NBA\nThe National Basketball Association, or NBA, is a professional basketball league compr',[-0.09, -0.0097])  , ()  ,   ,]
for chunk in chunks:
    response = client.embeddings.create(
        input=chunk,
        model="text-embedding-3-small"
    )
    embedded_chunks.append( (chunk, response.data[0].embedding) )
print(embedded_chunks)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [13]:
def cosine_simularity(A, B):
    return np.dot(A,B) / ( np.linalg.norm(A) * np.linalg.norm(B) )
   # -> ABcos/ A/|A| * B/|B|

In [14]:
question = 'Why was the nba made'

In [15]:
question_embedding = client.embeddings.create(
        input=question,
        model="text-embedding-3-small"
    ).data[0].embedding
print(question_embedding)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
best_cosine_similarity = 0
index = 0
for i in range(len(embedded_chunks)):
    similarity = cosine_simularity(question_embedding, embedded_chunks[i][1])
    if similarity > best_cosine_similarity:
        best_cosine_similarity = similarity
        index = i

In [ ]:
print(f"Relevant information: {embedded_chunks[index][0]}")

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user", "content": f"User question: {question}. Potentially useful information: {embedded_chunks[index][0]}"
        }
    ]
)

In [ ]:
print(completion.choices[0].message.content)